In [521]:
import math
import random
import numpy as np

In [522]:
OPTIMUS_PRIMES = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
R = {}

for d in OPTIMUS_PRIMES:
    R[d] = [1]
    while R[d].count(R[d][-1]) < 2:
        R[d].append((R[d][-1] * 2) % d)
    R[d].pop()
    print(R[d])

[1, 0]
[1, 2]
[1, 2, 4, 3]
[1, 2, 4]
[1, 2, 4, 8, 5, 10, 9, 7, 3, 6]
[1, 2, 4, 8, 3, 6, 12, 11, 9, 5, 10, 7]
[1, 2, 4, 8, 16, 15, 13, 9]
[1, 2, 4, 8, 16, 13, 7, 14, 9, 18, 17, 15, 11, 3, 6, 12, 5, 10]
[1, 2, 4, 8, 16, 9, 18, 13, 3, 6, 12]
[1, 2, 4, 8, 16, 3, 6, 12, 24, 19, 9, 18, 7, 14, 28, 27, 25, 21, 13, 26, 23, 17, 5, 10, 20, 11, 22, 15]
[1, 2, 4, 8, 16]
[1, 2, 4, 8, 16, 32, 27, 17, 34, 31, 25, 13, 26, 15, 30, 23, 9, 18, 36, 35, 33, 29, 21, 5, 10, 20, 3, 6, 12, 24, 11, 22, 7, 14, 28, 19]
[1, 2, 4, 8, 16, 32, 23, 5, 10, 20, 40, 39, 37, 33, 25, 9, 18, 36, 31, 21]
[1, 2, 4, 8, 16, 32, 21, 42, 41, 39, 35, 27, 11, 22]
[1, 2, 4, 8, 16, 32, 17, 34, 21, 42, 37, 27, 7, 14, 28, 9, 18, 36, 25, 3, 6, 12, 24]


In [523]:
def petod_drobnyx_mylen(num):
    b = bin(num)[:1:-1]
    
    if b[0] == '0':
        return 2
    
    for d in OPTIMUS_PRIMES[1::]:
        sum = 0
        for i in range(len(b)):
            sum += int(b[i]) * R[d][i % len(R[d])]
            sum %= d
        
        if sum == 0:
            return d

    return 1

petod_drobnyx_mylen(155)

5

In [524]:
def rho_f(x, mod):
    return (x**2 + 1) % mod

def rho_pollard(num, x_0 = 2, f = rho_f):
    X = [f(x_0, num)]
    Y = [f(X[0], num)]

    i = 0
    while X.count(X[i]) != 2:
        X.append(f(X[i], num))
        Y.append(f(f(Y[i], num), num))
        i += 1

        # print(f"{X[i], Y[i]}")
        if X[i] == Y[i]:
            return 1

        d = math.gcd(num, (Y[i] - X[i]) % num)
        if d != 1:
            return d

    return 1


rho_pollard(8051, x_0=2)

97

In [525]:
def jacobi(a, n):
    a = int(a)
    
    if math.gcd(a, n) != 1:
        return 0

    if a == 1:
        return 1

    if a > n:
        return jacobi(a % n, n)

    if a % 2 == 0:
        if n % 8 == 1 or n % 8 == 7:
            return jacobi(a / 2, n)
        else:
            return (-1) * jacobi(a / 2, n)

    if n % 4 == 1 or a % 4 == 1:
        return jacobi(n, a)
    else:
        return (-1) * jacobi(n, a)


jacobi(13, 31)

-1

In [526]:
def miller_rabin(num, base):
    i = 1
    while (num - 1) % (2 ** i) == 0:
        i += 1

    k = i - 1
    d = (num - 1) // (2 ** k)

    a_d = pow(base, d, num)

    if a_d == 1:
        return True
    
    a_d2i = a_d
    for j in range(k):
        if a_d2i == (num - 1):
            return True
        
        a_d2i = (a_d2i ** 2) % num

    return False


def check_prime(num, error_prob = 0.01):
    t = int(math.ceil(math.log(1 / error_prob, 4)))

    s = 0
    for _ in range(t):
        a = random.randrange(3, num + 1)
        s += int(miller_rabin(num, a))

    return s > (t / 2)


In [527]:
def add_columns(A, fr, to):
    if fr == to:
        return      # Smort optimi3aцion
    
    for i in range(len(A)):
        A[i][to] ^= A[i][fr]


# A - square matrix n x m
# returns rows with linear independed vectors
def gaussian_elim_gf2(A):
    m = len(A[0])
    n = len(A) - m
    marked = []                 # marked rows
    for j in range(m):          # cols
        for i in range(n):      # rows
            if A[i][j] == 1:
                marked.append(i)
                # eliminate 1s in this row
                for k in range(m):  
                    if A[i][k] == 1:
                        add_columns(A, j, k)
                break
    return marked

def get_ker_basis(A):
    n = len(A)
    m = len(A[0])
    # build matrix for Gauss elimination
    I = np.identity(m, dtype=int)
    B = np.vstack((A, I)) 
    _marked = gaussian_elim_gf2(B)
    B = np.transpose(B)

    ker_basis = []
    cmp_list = [0 for _ in range(n)]
    for i in range(m):
        if np.all(B[i][0:n:] == cmp_list):
            ker_basis.append(B[i][n::].tolist())

    return ker_basis

In [528]:
def sqeq_modp(a, b, c, p): # сквек
    k = (b**2 - 4*a*c) % p

    if jacobi(k, p) != 1:
        print("error")

    y = sqrt_modp(k, p)

    x = [0, 0]
    x[0] = (pow(2*a, -1, p) * (y[0] - b)) % p
    x[1] = (pow(2*a, -1, p) * (y[1] - b)) % p 

    return x


def sqrt_modp(a, p):
    if jacobi(a, p) != 1:
        print("error")

    if p % 4 == 3:
        # print("4k + 3")
        sq_a = pow(a, (p + 1) // 4, p)
        return [sq_a, p - sq_a]
    
    if p % 8 == 5:
        # print("8k + 5")
        k = (p - 5) // 8
        if pow(a, 2*k + 1, p) == 1:
            sq_a = pow(a, k + 1, p)
        else:
            sq_a = (pow(a, k + 1, p) * pow(2, 2*k + 1, p)) % p

        return[sq_a, p - sq_a]
    
    if p % 8 == 1:
        # print("8k + 1")
        b = 2
        while jacobi(b, p) != -1:
            b = random.randrange(3, p - 1)

        t_a = (p - 1) // 2
        t_b = 0

        while t_a % 2 == 0:
            if (pow(a, t_a, p) * pow(b, t_b, p)) % p  == p - 1:
                t_b += (p - 1) // 2

            t_a = t_a // 2
            t_b = t_b // 2

        if (pow(a, t_a, p) * pow(b, t_b, p)) % p  == p - 1:
                t_b += (p - 1) // 2

        sq_a = (pow(a, (t_a + 1) // 2, p) * pow(b, t_b // 2, p)) % p
        return[sq_a, p - sq_a]
            

sqrt_modp(943, 7793)
sqeq_modp(5, 32, 121, 7907)

[4902, 4580]

In [529]:
Prime_Base_Bound = 1000
Sieving_Interval = 100

def QS(num, B = Prime_Base_Bound, M = Sieving_Interval, MAX_SMOOTH=200):
    P = [3, 5, 7, 11]
    m = math.isqrt(num)

    def q(x):
        return ((x + m)**2 - num) % num
    
    for a in range(12, B):
        if check_prime(a):
            if jacobi(num, a) == 1:
                P.append(a)

    logP = [math.log2(p) for p in P]

    Q = []
    
    for x in range(-M, M):
        q_x = q(x)
        Q.append([x, q_x, math.log2(q_x)])

    for i in range(len(P)):
        x_1, x_2 = sqeq_modp(1, -2 * m, m**2 - num, P[i])

        for k in range(-(M // P[i]), (M // P[i])):
            pos_x1 = x_1 + k * P[i] + M
            pos_x2 = x_2 + k * P[i] + M
            Q[pos_x1][2] -= logP[i]
            Q[pos_x2][2] -= logP[i]
    
    Q = sorted(Q, key=lambda a: a[2])

    def smoothness(n):
        powers = [0]*len(P)
        for i in range(len(P)):
            while n % P[i] == 0:
                powers[i] += 1
                n //= P[i]
        
        if n != 1:
            return []
        return powers
    
    Smooth = {}

    i = 0
    for [_x, qx, logq] in Q:
        if i >= MAX_SMOOTH:
            break
        pows = smoothness(qx)
        if len(pows) != 0:
            Smooth[qx] = pows
            i += 1

    A = [[i % 2 for i in v] for v in Smooth.values()]
    A = np.transpose(A)
    ker = get_ker_basis(A)
    print(len(ker))
    for v in ker:
        print(v)
    
    S = {}

    

    
num = 323324583518541583
QS(num, B=10000, M=100000)

error
error
error
error
error
error
41
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,